<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании. Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, и обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель в данной задаче не требуется.

## План работы

1) Загрузить и изучить данные.
2) Ответить на вопрос и обосновать решение: 
 Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?
 - Изменится. Привести примеры матриц.
 - Не изменится. Указать, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
3) Предложить алгоритм преобразования данных для решения задачи. Обосновать, почему качество линейной регрессии не поменяется.
4) Запрограммировать этот алгоритм, применив матричные операции. Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применять метрику R2.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

Изучим данные:

In [3]:
print(df.head())

   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Изменим типы данных, где необходимо:

In [5]:
df['Возраст'] = df['Возраст'].astype('int')
df['Зарплата'] = df['Зарплата'].astype('int')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


Проверим на дубликаты:

In [7]:
df[df.duplicated()]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39,48100,1,0
488,1,24,32900,1,0
513,0,31,37400,2,0
718,1,22,32600,1,0
785,0,20,35800,0,0
...,...,...,...,...,...
4793,1,24,37800,0,0
4902,1,35,38700,1,0
4935,1,19,32700,0,0
4945,1,21,45800,0,0


Так как мы не знаем, соответствует ли распределение выборки распределению генеральной совокупности, оставим дубликаты.

Выведем статистику по данным:

In [8]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.359400,1.194200,0.148000
std,0.500049,8.440807,9900.082063,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Видим, что аномалий в данных нет.

## Умножение матриц

Разработаем алгоритм шифрования и докажем его эффективность:

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество не должно измениться, т. к. предсказания у двух моделей будут одинаковыми, т. е. $XPw_1 = Xw$

**Обоснование:**
1. Найдем веса линейной регрессии на зашифрованных данных \
$X P w_1 = y_1$\
$(XP)^T X P w_1 = (XP)^T y_1$\
$w_1 = ((XP)^T XP)^{-1} (XP)^T y_1$
2. Преобразуем полученное выражение \
$w_1 = (P^T X^T XP)^{-1} P^T X^T y_1$\
$w_1 = ((P^T X^T X) P)^{-1} P^T X^T y_1$\
$w_1 = P^{-1} (P^T X^T X)^{-1} P^T X^T y_1$\
$w_1 = P^{-1} (P^T (X^T X))^{-1} P^T X^T y_1$\
$w_1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y_1$\
$w_1 = P^{-1} (X^T X)^{-1} I X^T y_1$\
$w_1 = P^{-1} (X^T X)^{-1} X^T y_1$
3. Выведем равенство $X_1 w_1 = Xw$, где $X_1 = XP, y = y_1$\
$X_1 w_1 = XPw_1 = XP P^{-1} (X^T X)^{-1} X^T y_1$\
$X_1 w_1 = X I (X^T X)^{-1} X^T y_1$\
$X_1 w_1 = X (X^T X)^{-1} X^T y_1 = Xw$

Размерности весов исходной задачи и весов, полученных из зашифрованных данных, одинаковы.

## Алгоритм преобразования

**Алгоритм**

Для создания зашифрованных данных умножим справа исходный датафрейм на случайную квадратную матрицу P, полученных из данных нормального распределения.
Далее рассчитаем предсказания на исходных и зашифрованных данных, рассчитаем метрики R2 и сравним их.

**Обоснование**

Данная трансформация не должна привести к снижению качества, исходя из п. 2.

## Проверка алгоритма

In [9]:
X = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']

# создадим случайную матрицу
P = np.random.normal(size=(X.shape[1], X.shape[1]))

# проверим матрицу на обратимость
try:
    P_inverse = np.linalg.inv(P)
    print('Матрица обратима')
except:
    print('Матрица необратима')

Матрица обратима


Разобьем датасет на выборки и умножим признаки на обратимую матрицу.

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=0)
X_train_encrypted = X_train @ P
X_valid_encrypted = X_valid @ P

Создадим модель линейной регрессии и обучим ее.

Модель до шифрования:

In [11]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_valid)
score = r2_score(y_valid, predictions)
print(score.round(4))

0.3879


Модель после шифрования:

In [12]:
model_encrypted = LinearRegression()
model_encrypted.fit(X_train_encrypted, y_train)
predictions_encrypted = model_encrypted.predict(X_valid_encrypted)
score_encrypted = r2_score(y_valid, predictions_encrypted)
print(score_encrypted.round(4))

0.3879


Мы видим, что качество модели не изменилось.

## Вывод

Мы разработали алгоритм шифрования данных и математически доказали, что эффективность модели линейной регрессии на зашифрованных данных не изменится по сравнению с исходными.
Мы также обучили модель на датасете до и после шифрования и убедились в идентичности результатов, применив метрику R2.